In [13]:
import pandas as pd
import numpy as np
from glob import glob
import numbers

In [14]:
filenames = glob('Combined_*[a,b,c,d,e,f,g,h,i,j,k]*160c.dta')
list_dfs  = [pd.read_stata(f) for f in filenames]

In [15]:
list_dfs = [list_dfs[i].set_index('xwaveid') for i in range(11)]

In [19]:
list_males = [list_dfs[i][list_dfs[i].filter(regex='hgsex$').columns]for i in range(11)]
list_males = [list_dfs[i][list_males[i].iloc[:,0] == '[1] Male'] for i in range(11)]

In [49]:
waves_employed_index = [list_dfs[i].filter(regex='esdtl$').columns for i in range(11)] #
waves_employed = [list_dfs[i][waves_employed_index[i]] for i in range(11)] #waves_employed[0-10] are now created (waves1-11)

In [51]:
waves_employed[0]['aesdtl']

xwaveid
0100001    [6] Not in the labour force, not marginally at...
0100002                                      [2] Employed PT
0100003                                      [2] Employed PT
0100004                                      [2] Employed PT
0100005                                      [2] Employed PT
                                 ...                        
0119910                          [-10] Non-responding person
0119911                          [-10] Non-responding person
0119912                                      [1] Employed FT
0119913                          [-10] Non-responding person
0119914                          [-10] Non-responding person
Name: aesdtl, Length: 19914, dtype: category
Categories (7, object): [[-10] Non-responding person < [1] Employed FT < [2] Employed PT < [3] Unemployed, looking for FT work < [4] Unemployed, looking for PT work < [5] Not in the labour force, marginally attached < [6] Not in the labour force, not marginally at...]

In [52]:
males_employed = [list_males[i][waves_employed[i].iloc[:,0] == '[1] Employed FT'] for i in range(11)]

C:\Users\idias\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [59]:
pref1 = [males_employed[i].filter(regex='jbhrcpr$') for i in range(11)]
pref2 = [males_employed[i].filter(regex='jbhrcpr$') for i in range(11)]
pref2 = pref2[1:]

In [60]:
pref1[0]

,ajbhrcpr
xwaveid,
0100014,[1] Fewer hours
0100019,[1] Fewer hours
0100022,[1] Fewer hours
0100024,[2] About the same
0100028,[2] About the same
...,...
0119892,[1] Fewer hours
0119893,[3] More hours
0119903,[3] More hours


In [61]:
diff = [pref1[i].join(pref2[i]) for i in range(10)]
cleaned = [diff[i].dropna(how = 'any') for i in range(10)]

In [62]:
cleaned[0]

,ajbhrcpr,bjbhrcpr
xwaveid,,
0100014,[1] Fewer hours,[1] Fewer hours
0100019,[1] Fewer hours,[1] Fewer hours
0100022,[1] Fewer hours,[1] Fewer hours
0100024,[2] About the same,[2] About the same
0100030,[1] Fewer hours,[2] About the same
...,...,...
0119863,[2] About the same,[2] About the same
0119865,[2] About the same,[1] Fewer hours
0119876,[3] More hours,[3] More hours


In [63]:
t_minus_1 = [cleaned[i].iloc[:,0].value_counts() for i in range(10)]
t = [cleaned[i].iloc[:,1].value_counts() for i in range(10)]

In [64]:
clean_fewer = [cleaned[i][cleaned[i].iloc[:,0] == '[1] Fewer hours'] for i in range(10)]
clean_fewer = [clean_fewer[i].iloc[:,1].value_counts(normalize=True) for i in range(10)]

clean_current = [cleaned[i][cleaned[i].iloc[:,0] == '[2] About the same']for i in range(10)]
clean_current = [clean_current[i].iloc[:,1].value_counts(normalize=True) for i in range(10)]

clean_more = [cleaned[i][cleaned[i].iloc[:,0] == '[3] More hours'] for i in range(10)]
clean_more = [clean_more[i].iloc[:,1].value_counts(normalize=True) for i in range(10)]

In [65]:
clean_fewer = pd.DataFrame(clean_fewer)
clean_current = pd.DataFrame(clean_current)
clean_more  = pd.DataFrame(clean_more)

In [66]:
clean_unfin = clean_fewer.merge(clean_current, left_index=True, right_index=True)
clean_merged = clean_unfin.merge(clean_more, left_index=True, right_index=True)

In [67]:
unclean = clean_merged.mean(axis=0)
unclean = unclean*100

In [68]:
#_X = prefer to work fewer hours, _Y = current hours are good, _ = Prefer to work more hours

In [69]:
unclean = pd.DataFrame(unclean)

In [70]:
fewer_hrs = unclean[:7]
curr_hrs = unclean[7:14]
more_hrs = unclean[14:]

In [71]:
fewer_hrs.columns = ['Prefer to work fewer hours (t-1)']
curr_hrs.columns = ['Current hours are good (t-1)']
more_hrs.columns = ['Prefer to work more hours (t-1)']

In [72]:
fewer_hrs.index = more_hrs.index
curr_hrs.index = more_hrs.index

In [73]:
final_table = fewer_hrs.join(curr_hrs)
final_table = final_table.join(more_hrs)

In [74]:
final_table.sort_values('Prefer to work fewer hours (t-1)', ascending=False)

,Prefer to work fewer hours (t-1),Current hours are good (t-1),Prefer to work more hours (t-1)
[1] Fewer hours,64.497736,20.847067,10.825055
[2] About the same,33.512254,73.416013,49.660006
[3] More hours,1.952059,5.652767,39.467546
[-3] Dont know,0.037952,0.077914,0.047393
[-10] Non-responding person,0.000000,0.000000,0.000000
[-1] Not asked,0.000000,0.000000,0.000000
[-4] Refused/Not stated,0.000000,0.012477,0.000000


Small extension: The above table is an average of ALL waves (t-1). 